# How to use TensorBoard in Colab
## Download and unzip ngrok

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-04-24 08:02:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.235.130.91, 23.23.171.5, 50.19.252.69, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.235.130.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.88MB/s    in 0.6s    

2018-04-24 08:02:46 (8.88 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL
Run the next cell to start the training before open the url.

In [6]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://ed3f368a.ngrok.io


## Run a Keras model with TenorBoard

In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 200us/step - loss: 0.2600 - acc: 0.9202 - val_loss: 0.0556 - val_acc: 0.9810
Epoch 2/12
23808/60000 [==========>...................] - ETA: 6s - loss: 0.0924 - acc: 0.9723

60000/60000 [==============================] - 11s 182us/step - loss: 0.0877 - acc: 0.9745 - val_loss: 0.0392 - val_acc: 0.9871
Epoch 3/12
60000/60000 [==============================] - 11s 182us/step - loss: 0.0665 - acc: 0.9795 - val_loss: 0.0347 - val_acc: 0.9880
Epoch 4/12
52992/60000 [=========================>....] - ETA: 1s - loss: 0.0537 - acc: 0.9839

60000/60000 [==============================] - 11s 181us/step - loss: 0.0539 - acc: 0.9839 - val_loss: 0.0305 - val_acc: 0.9899
Epoch 5/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0465 - acc: 0.9860 - val_loss: 0.0341 - val_acc: 0.9891
Epoch 6/12
60000/60000 [==============================] - 11s 183us/step - loss: 0.0421 - acc: 0.9873 - val_loss: 0.0276 - val_acc: 0.9904
Epoch 7/12
  384/60000 [..............................] - ETA: 13s - loss: 0.0266 - acc: 0.9896

60000/60000 [==============================] - 11s 183us/step - loss: 0.0377 - acc: 0.9888 - val_loss: 0.0307 - val_acc: 0.9902
Epoch 8/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0349 - acc: 0.9896 - val_loss: 0.0294 - val_acc: 0.9902
Epoch 9/12
47488/60000 [======================>.......] - ETA: 2s - loss: 0.0331 - acc: 0.9902

60000/60000 [==============================] - 11s 183us/step - loss: 0.0325 - acc: 0.9902 - val_loss: 0.0316 - val_acc: 0.9909
Epoch 10/12
60000/60000 [==============================] - 11s 184us/step - loss: 0.0306 - acc: 0.9907 - val_loss: 0.0292 - val_acc: 0.9903
Epoch 11/12
57472/60000 [===========================>..] - ETA: 0s - loss: 0.0290 - acc: 0.9913

60000/60000 [==============================] - 11s 186us/step - loss: 0.0294 - acc: 0.9912 - val_loss: 0.0241 - val_acc: 0.9924
Epoch 12/12
60000/60000 [==============================] - 11s 183us/step - loss: 0.0265 - acc: 0.9919 - val_loss: 0.0276 - val_acc: 0.9907
Test loss: 0.02756775512016866
Test accuracy: 0.9907


## Optionally check tensorboard and ngrok PIDs and stop them

In [8]:
!ps aux | grep tensorboard

root       155  0.3  5.1 39898084 685952 ?     Sl   08:12   0:16 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root       184  107  0.0  33960  5000 pts/0    Ss+  09:37   0:01 /bin/sh -c ps aux | grep tensorboard
root       186  0.0  0.0  38196  5612 pts/0    S+   09:37   0:00 grep tensorboard


In [7]:
!ps aux | grep ngrok

root       158  0.2  0.1  19512 13724 ?        Sl   08:12   0:11 ./ngrok http 6006
root       181  108  0.0  33964  4980 pts/0    Ss+  09:37   0:01 /bin/sh -c ps aux | grep ngrok
root       183  0.0  0.0  38200  5016 pts/0    S+   09:37   0:00 grep ngrok


In [0]:
!kill 155

## Optionally clear the tensorboard records directory

In [0]:
!rm -rf log